In [1]:
import numpy as np
from nltk.tokenize import word_tokenize
import pandas as pd
import pickle
from tqdm import tqdm_notebook as tqdm
import nltk.translate.bleu_score as bleu
from difflib import SequenceMatcher

In [2]:
all_df = pd.read_pickle('data/rule_based_corrected_df.pkl')
all_df.head()

,Original,Target 0,Target 1,Target 2,Target 3,Category,Dataset
0,I mean that you have to really be her friend.,And I mean Really be her friend.,Just be her BFF 4 real.,you have to be her friend.,"You have to actually be her friend, for real.",Family_Relationships,test
1,Are you posing a rhetorical question?,Sounds like a rhetorical question :),Do you really want an answer?,That sounds more like a rhetorical question th...,Are you asking me a rhetorical question?,Family_Relationships,test
2,Men pretend to love in order to have intercour...,"Men play at love to get sex, women play at sex...","Men fake love to get laid, women fake orgasms ...","Guys PRETEND to love so they can get laid, wom...",Dudes just act like they love a chick to get b...,Family_Relationships,test
3,I do not intend to be mean.,I don't want to be mean.,I wasn't trying to be a jerk.,I'm not tryin to be mean...,I didn't want to be mean,Family_Relationships,test
4,I would estimate an average of 45% initially b...,On average I'd say about 45% at first but than...,"It's a little less than 50/50 at the start, bu...",Prolly 45% at the start but when you get to no...,"I guess it'd be around 45% to start with, but ...",Family_Relationships,test


In [6]:
results_df = pd.read_pickle('data/v1_results.pkl')[['Source', 'Target', 'Prediction', 'Copy Prediction', 'No Entity Copy Prediction']]
results_df.head()

,Source,Target,Prediction,Copy Prediction,No Entity Copy Prediction
0,I mean that you have to really be her friend .,And I mean Really be her friend .,I mean you have to really be her friend .,I mean you have to really be her friend .,I mean you have to be her friend .
1,Are you posing a rhetorical question ?,Sounds like a rhetorical question : ),What kind of question is that ?,What are you asking a question ?,What kind of question is that ?
2,Men pretend to love in order to have intercour...,"Men play at love to get sex , women play at se...","Men play love to have sex , women play for sex...","Men pretend to love in order to have sex , wom...","Men play to love in order to have sex , women ..."
3,I do not intend to be mean .,I do n't want to be mean .,I do n't mean to be mean .,I do n't mean to be mean .,I do n't mean to be mean .
4,I would estimate an average of 45 % initially ...,On average I 'd say about 45 % at first but th...,"I would say a 15 % of 45 % , then once you get...","45 % of them , but once you get to know the pe...",45 % of 45 % and then after you know the perso...


In [4]:
len(results_df)

2101

In [137]:
for idx, row in results_df.sample(1).iterrows():
    print('Source:\t\t' + row['Source'])
    print('Target:\t\t' + row['Target'])
    print('Pred:\t\t' + row['Prediction'])
    print('Copy Pred:\t' + row['Copy Prediction'])
    print('No ENT Pred:\t' + row['No Entity Copy Prediction'])

Source:		I behaved as such in the company of my present romantic partner .
Target:		I did this with my current boyfriend .
Pred:		I like my boyfriend and I have a boyfriend .
Copy Pred:	I used to love my boyfriend .
No ENT Pred:	I used to like my boyfriend .


# BLEU

In [7]:
test_tgt0 = list(all_df[all_df['Dataset'] == 'test']['Target 0'])
test_tgt1 = list(all_df[all_df['Dataset'] == 'test']['Target 1'])
test_tgt2 = list(all_df[all_df['Dataset'] == 'test']['Target 2'])
test_tgt3 = list(all_df[all_df['Dataset'] == 'test']['Target 3'])
assert(len(test_tgt0) == len(test_tgt1) == len(test_tgt2) == len(test_tgt3))

ref = []
for idx in range(len(test_tgt0)):
    ref.append([test_tgt0[idx], test_tgt1[idx], test_tgt2[idx], test_tgt3[idx]])

hyp = list(results_df['Prediction'])

In [8]:
print('BLEU scores using all 4 targets.')
bleu.corpus_bleu(ref, hyp)

BLEU scores using all 4 targets.


0.7395791986703688

In [9]:
# try with only 1 target - why is it otherwise so good?
print('BLEU scores using only one of 4 targets.')
print(bleu.corpus_bleu([[line] for line in test_tgt0], hyp))
print(bleu.corpus_bleu([[line] for line in test_tgt1], hyp))
print(bleu.corpus_bleu([[line] for line in test_tgt2], hyp))
print(bleu.corpus_bleu([[line] for line in test_tgt3], hyp))

BLEU scores using only one of 4 targets.
0.5238716701257187
0.4483937207801788
0.44976733715218586
0.45597632782422576


In [13]:
# how close are the predicted to the original?
test_orig = list(results_df['Source'])
print('BLEU score of source data to prediction')
print(bleu.corpus_bleu([[line] for line in test_orig], hyp))

BLEU score of source data to prediction
0.6206195843828992


In [14]:
print(bleu.corpus_bleu(ref, [line for line in test_orig]))

0.6995504029544203


In [15]:
val_tgt0 = list(all_df[all_df['Dataset'] == 'tune']['Target 0'])
val_tgt1 = list(all_df[all_df['Dataset'] == 'tune']['Target 1'])
val_tgt2 = list(all_df[all_df['Dataset'] == 'tune']['Target 2'])
val_tgt3 = list(all_df[all_df['Dataset'] == 'tune']['Target 3'])
assert(len(val_tgt0) == len(val_tgt1) == len(val_tgt2) == len(val_tgt3))

ref_val = []
for idx in range(len(val_tgt0)):
    ref_val.append([val_tgt0[idx], val_tgt1[idx], val_tgt2[idx], val_tgt3[idx]])
    
pred_file_val = open('data/pred-val.txt')
hyp_val = [line.rstrip() for line in pred_file_val.readlines()]
pred_file_val.close()

In [16]:
bleu.corpus_bleu(ref_val, hyp_val)

0.7795881685210191

# Similarity Scores & differences between default and copy models

In [17]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [18]:
similar("I mean that you have to really be her friend.", "And I mean Really be her friend.")

0.7012987012987013

In [20]:
similar("I mean that you have to really be her friend.", "I mean you have to really be her friend .")

0.9302325581395349